In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

In [ ]:
mean = [0.49159187, 0.48234594, 0.44671956]
std = [0.23834434, 0.23486559, 0.25264624]

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='datasets/cifar10/train',
                                        train=True,
                                        download=True,
                                        transform=train_transform)

In [ ]:
testset = torchvision.datasets.CIFAR10(root='datasets/cifar10/train',
                                       train=False,
                                       download=True,
                                       transform=test_transform)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=16,
                                          shuffle=True,
                                          num_workers=2)

In [ ]:
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=16,
                                         shuffle=False,
                                         num_workers=2)

In [ ]:
in_size = 3
hid1_size = 16
hid2_size = 32
hid3_size = 64

out1_size = 2000
out2_size = 10

k_conv_size = 9

In [ ]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()

        self.layer1 = nn.Sequential(
                                    nn.Conv2d(in_size, hid1_size, k_conv_size),
                                    nn.BatchNorm2d(hid1_size),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2))

        self.layer2 = nn.Sequential(
                                    nn.Conv2d(hid1_size, hid2_size, k_conv_size),
                                    nn.BatchNorm2d(hid2_size),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer3 = nn.Sequential(
                                    nn.Conv2d(hid2_size, hid3_size, k_conv_size),
                                    nn.BatchNorm2d(hid3_size),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2))

        self.layer4 = nn.Sequential(
                                    nn.Linear(hid3_size * k_conv_size * k_conv_size, out1_size),
                                    nn.ReLU(), 
                                    nn.Dropout(0.5),
                                    nn.Linear(out1_size, out2_size))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.layer4(out)

        return F.log_softmax(out, dim=-1)


In [ ]:
model = ConvNet()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model.to(device)

In [ ]:
learning_rate = 0.001
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_step = len(trainloader)
num_epochs = 10
loss_values = list()

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
            loss_values.append(loss.item())

print('Finished Training')